In [10]:

''' 
1. Create hadoop architecture diagram.


2. Create hive architecture diagram.


3. What is difference between dynamic partition vs static partition?
static partition -- we load data files into table partition directly via load command
dynamic partition -- use column name create partition and insert data

4. How to explain plan of hive query?

expression tree

EXPLAIN
FROM src INSERT OVERWRITE TABLE dest_g1 SELECT src.key, sum(substr(src.value,4)) GROUP BY src.key;

STAGE PLANS:
  Stage: Stage-1
    Map Reduce
      Alias -> Map Operator Tree:
        src
            Reduce Output Operator
              key expressions:
                    expr: key
                    type: string
              sort order: +
              Map-reduce partition columns:
                    expr: rand()
                    type: double
              tag: -1
              value expressions:
                    expr: substr(value, 4)
                    type: string
      Reduce Operator Tree:
        Group By Operator
          aggregations:
                expr: sum(UDFToDouble(VALUE.0))
          keys:
                expr: KEY.0
                type: string
          mode: partial1
          File Output Operator
            compressed: false
            table:
                input format: org.apache.hadoop.mapred.SequenceFileInputFormat
                output format: org.apache.hadoop.mapred.SequenceFileOutputFormat
                name: binary_table
 
  Stage: Stage-2
    Map Reduce
      Alias -> Map Operator Tree:
        /tmp/hive-zshao/67494501/106593589.10001
          Reduce Output Operator
            key expressions:
                  expr: 0
                  type: string
            sort order: +
            Map-reduce partition columns:
                  expr: 0
                  type: string
            tag: -1
            value expressions:
                  expr: 1
                  type: double
      Reduce Operator Tree:
        Group By Operator
          aggregations:
                expr: sum(VALUE.0)
          keys:
                expr: KEY.0
                type: string
          mode: final
          Select Operator
            expressions:
                  expr: 0
                  type: string
                  expr: 1
                  type: double
            Select Operator
              expressions:
                    expr: UDFToInteger(0)
                    type: int
                    expr: 1
                    type: double
              File Output Operator
                compressed: false
                table:
                    input format: org.apache.hadoop.mapred.TextInputFormat
                    output format: org.apache.hadoop.hive.ql.io.IgnoreKeyTextOutputFormat
                    serde: org.apache.hadoop.hive.serde2.dynamic_type.DynamicSerDe
                    name: dest_g1
 
  Stage: Stage-0
    Move Operator
      tables:
            replace: true
            table:
                input format: org.apache.hadoop.mapred.TextInputFormat
                output format: org.apache.hadoop.hive.ql.io.IgnoreKeyTextOutputFormat
                serde: org.apache.hadoop.hive.serde2.dynamic_type.DynamicSerDe
                name: dest_g1


there ev_vehicle datasets in data/hive folder.
1. Create a external table to store the data.  [Table name: ev_vehicle_info_src]
''' 
# HIVE #######################################################
# hive -f sql/cards_ingest/ct_vehicle_info_src.hql
# hive -e "load data inpath 's3://quintrix-spearscjs/cards_ingest/ev_vehicle_info_src/Electric_Vehicle_Population_Data.csv' into table cards_ingest.ev_vehicle_info_src;"


# ATHENA #####################################################
'''
create external table if not exists cards_ingest.ev_vehicle_info_src_athena ( 
      vin varchar(10), county varchar(30), city varchar(30), state varchar(2), postal_code varchar(5), model_year int, 
      make string, model string, electric_vehicle_type string, cafv_eligibility string, electric_range int, base_msrp decimal(12,2), 
      legislative_district string, dol_vehicle_id string, vehicle_location string, electric_utility string, census_tract_msrp_2020 string ) 
row format delimited fields terminated by ',' location "s3://quintrix-spearscjs/data/cards_ingest/ev_vehicle_info_src/" tblproperties ("skip.header.line.count"="1")
'''

17
19


In [ ]:
''' 
2. Create another table with partition make.  [ Parquet table with snappy compression][Table name:ev_vehicle_info]

'''
# HIVE #######################################################
# hive -f sql/cards_ingest/ct_vehicle_info.hql
# q="insert into cards_ingest.ev_vehicle_info partition(make) select vin, county,city,state,postal_code,model_year,model,electric_vehicle_type,
#     cafv_eligibility,electric_range,base_msrp,legislative_district,dol_vehicle_id,vehicle_location,electric_utility ,census_tract_msrp_2020, make 
#     from cards_ingest.ev_vehicle_info_src"
# hive -e $q 


# ATHENA #####################################################
''' 
CREATE TABLE cards_ingest.ev_vehicle_info_athena 
WITH(location='s3://quintrix-spearscjs/athena/cards_ingest/ev_vehicle_info_athena/', 
    partitioned_by=ARRAY['make'], format = 'PARQUET', parquet_compression = 'SNAPPY') 
AS SELECT vin,county,city,state,postal_code,model_year,model,electric_vehicle_type,
     cafv_eligibility,electric_range,base_msrp,legislative_district,dol_vehicle_id,vehicle_location,electric_utility ,
     census_tract_msrp_2020, make 
FROM ev_vehicle_info_src_athena;
'''

In [ ]:
''' 
3. Write a sql to show top 3 by number of vehicle registered. [ sql should be in file. the number to pas is dynamic. it can be 3 is can be 5]
'''
# hive -f sql/cards_ingest/template/top_n_ev_vehicle_info.sql -hivevar n=5

In [ ]:
''' 
4. Find out the vehicle which are Not eligible due to low battery range='Not eligible due to low battery range'
''' 
# hive -e "select vin, county, city, state, postal_code, model_year, model, cafv_eligibility from cards_ingest.ev_vehicle_info where cafv_eligibility='Not eligible due to low battery range';"

In [ ]:
''' 
# 5. Write a sql to show top 3 by Write a sql to show top 3 by n (in sql file parameter top n) (take out for now)
'''

In [ ]:
'''
6. write a sql to show how many distinct model for each maker.
''' 
# hive -e 'select make, count(distinct model) num_distinct_models from cards_ingest.ev_vehicle_info GROUP BY make;'

In [ ]:
''' 
7.Write sql to count number of vehicle where Model Year < 2018 and Not eligible due to low battery range='Not eligible due to low battery range'
''' 
# hive -e "select count(*) from cards_ingest.ev_vehicle_info WHERE CAST(model_year as INT) < 2018 AND  cafv_eligibility='Not eligible due to low battery range';"

In [ ]:
''' 
AFTER DO THIS DO THE SAME THING IN ATHENA
'''

In [ ]:
# OLD FILE FOR CLASS QUESTION 2 BELOW

In [ ]:
from utils import db
from utils import s3
import pandas as pd
import random
import datetime

In [2]:

'''
1.Create table :tran_fact
tran_id int, cust_id varchar(20),tran_ammount decimal(10,2), tran_type varchar(1), tran_country_cd varchar(3),tran_date date
tran_id : random id (ex : FX_123456,TD_224452  (Concat [FX/TD] with 6 digit number)
cust_id : cust_12345 (concat cust+ 4 digit number)
tran_ammount : $ ammount ( 3 dgit to 10 digit number)
tran_type : C/D (Random Value)
country_cd : ['USA','CAN','IND','AFG','CHN','JPN','KON','PAL']
tran_date : ['2022-01-01' to '2022-03-10' ]

Generate records for every day with the date range given [ each day generate 1000 to 10000 records]
have 7 days fro now in tester
'''

"\n1.Create table :tran_fact\ntran_id int, cust_id varchar(20),tran_ammount decimal(10,2), tran_type varchar(1), tran_country_cd varchar(3),tran_date date\ntran_id : random id (ex : FX_123456,TD_224452  (Concat [FX/TD] with 6 digit number)\ncust_id : cust_12345 (concat cust+ 4 digit number)\ntran_ammount : $ ammount ( 3 dgit to 10 digit number)\ntran_type : C/D (Random Value)\ncountry_cd : ['USA','CAN','IND','AFG','CHN','JPN','KON','PAL']\ntran_date : ['2022-01-01' to '2022-03-10' ]\n\nGenerate records for every day with the date range given [ each day generate 1000 to 10000 records]\nhave 7 days fro now in tester\n"

In [28]:
""" 
CREATE SCHEMA IF NOT EXISTS hadoop;
DROP TABLE hadoop.tran_fact;
CREATE TABLE IF NOT EXISTS hadoop.tran_fact(
    tran_id varchar(18) encode raw, 
    cust_id varchar(20) encode raw,
    tran_ammount decimal(10,2) encode AZ64, 
    tran_type varchar(1) encode raw, 
    tran_country_cd varchar(3) encode bytedict,
    tran_date date encode delta32k
);
"""

# def generate_dummy_data(start: datetime, num_days : int, num_records : int ):
    
start_date = datetime.datetime(2022, 1, 1)
num_days = 7
num_records = 5000


def generate_dummy_data(start: datetime, num_days : int, num_records : int, filepath = 'data/hadoop/'):
    tran_country_cds = ['USA','CAN','IND','AFG','CHN','JPN','KON','PAL']
    tran_types = ['C', 'D']
    id_prefixes = ['FX', 'TD']
    header = 'tran_id,cust_id,tran_ammount,tran_type,tran_country_cd,tran_date\n'
    files_created = []
    for i in range(num_days):
        tran_date = start_date + datetime.timedelta(days=i)
        filename = tran_date.strftime("tran_fact_%Y-%m-%d") + ".csv"
        f = open(filepath + filename, 'w')
        f.write(header)
        for j in range(num_records):
            tran_ammount = round(random.uniform(100.00, 99999999.99),2)
            tran_type = random.choice(tran_types)
            tran_country_cd = random.choice(tran_country_cds)
            tran_id = random.choice(id_prefixes) + '_' + tran_date.strftime("%Y%m%d") + '_' +  f"{j+1:06}"
            cust_id = 'cust_' + f"{random.randint(1, 9999):04}"
            f.write(tran_id + "," + cust_id + "," + str(tran_ammount) + ","
                + tran_type + "," + tran_country_cd + "," + tran_date.strftime("%Y-%m-%d") + '\n')
        files_created.append((filename, filepath, tran_date.strftime('%Y-%m-%d')))
        f.close()
    return files_created
        
files_created = generate_dummy_data(start_date, num_days, num_records)

In [29]:
''' 
2. Load each file into s3. [ each with folder as 2022-01-01,2022-01-02]
'''
for f in files_created:
    s3.upload_file(f[1] + f[0], 'quintrix-spearscjs', 'cards_ingest/tran_fact/daily/' + f[2] + '/' + f[0])

In [6]:
'''
3.Create table in Redshift.
'''
query = 'CREATE SCHEMA IF NOT EXISTS hadoop'
db.do_query(query, [])
db.do_query_file('sql/create/ct_hadoop_tran_fact.sql',[])

sql/create/ct_hadoop_tran_fact.sql done.


In [7]:
'''
4. Copy data to redshift  [append mode]
'''
db.copy('tran_fact', 'hadoop',  's3://quintrix-spearscjs/hadoop/data/daily/', db.redshift_iam, ',')

In [10]:
'''
1. Unload query sum(tran)amt by country cd to parquet file in s3.
'''
subquery = 'SELECT tran_country_cd, SUM(tran_ammount) FROM hadoop.tran_fact GROUP BY tran_country_cd;'
bucket = 's3://quintrix-spearscjs/hadoop/data/country/tran_ammt_sum/'
query = """ 
    UNLOAD (%s) -- query string
    TO %s   -- s3 bucket_name
    IAM_ROLE %s -- redshift iam role
    PARALLEL ON
    ALLOWOVERWRITE 
    FORMAT PARQUET
"""
db.do_query(query, [subquery, bucket, db.redshift_iam])